In [ ]:
%matplotlib inline

# Create rail track table from ways, with unwind order and lengths 🚃 🚃 🚃

This notebook assembles a table of rail tracks from already derived ways, determining track components and paths (unwind order) and track lengths.  

**Derived railtracks and railtracksparking is dropped and re-inserted!** 

Created on:  2016-12-08  
Last update: 2017-01-22  
Contact: michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [ ]:
cityname = "berlin"

modes = ["tram", "light_rail", "rail", "subway", "narrow_gauge", "funicular", "monorail"]

### Imports

In [ ]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
import pprint
pp = pprint.PrettyPrinter(indent=4)
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry

import osmnx as ox
import networkx as nx
import json
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from functools import partial
import pyproj
from scipy import spatial
from haversine import haversine

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

### DB Connection

In [ ]:
client = MongoClient()
db_raw = client[cityname+'_raw']
ways_raw = db_raw['ways']
nodes_raw = db_raw['nodes']
db_derived = client[cityname+'_derived']
railtracks = db_derived['railtracks']
railtracksparking = db_derived['railtracksparking']
ways_derived = db_derived['ways']

### Functions

In [ ]:
def file_len(fname): # http://stackoverflow.com/questions/845058/how-to-get-line-count-cheaply-in-python
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

def assembleRailTrack(ways_raw, ways_derived, nodes_raw, wayid, drawit = False, debug = False):
    oxelements = []
    elemnodesint = set()
    elemnodesint_real = set()
    elemways = []
    elemnodes = []
    nodesdict = {}
    
    for i,way in enumerate(ways_raw.find({"_id": wayid})):
        for n in way["nodes"]:
            elemnodesint.add(int(n))

    for n in elemnodesint:
        for nd in nodes_raw.find({"_id": n}):
            elemnodesint_real.add(int(nd["_id"]))
            elemnodes.append({"id": int(nd["_id"]), "lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0], "type": "node"})
            nodesdict[str(int(nd["_id"]))] = {"lat": nd["loc"]["coordinates"][1], "lon": nd["loc"]["coordinates"][0]}

    tracklength = 0
    for i,way in enumerate(ways_raw.find({"_id": wayid})):
        wayinfo = ways_derived.find_one({"_id":int(way["_id"])})
        waylength = wayinfo["properties_derived"]["length"]
        tracklength += waylength
        elemways.append({"id": int(way["_id"]), "nodes":[int(way["nodes"][k]) for k in range(len(way["nodes"])) if int(way["nodes"][k]) in elemnodesint_real], "tags": way["tags"], "type": "way"})

    oxelements = [{"elements": elemnodes + elemways}]
    if debug:
        # Check if nodes are missing
        if len(elemnodesint_real) < len(elemnodesint):
            print(str(len(elemnodesint)-len(elemnodesint_real)) + " nodes are missing.")
        print("oxelements:")
        pp.pprint(oxelements)
        
    G = ox.create_graph(oxelements, retain_all=True)
    if drawit:
        fig, ax = ox.plot_graph(G)
    G = nx.Graph(G)
    G = G.to_undirected()
    ccs = list(nx.connected_component_subgraphs(G))

    components = []
    for c in range(len(ccs)):
        deglist = np.array(list((ccs[c].degree_iter())))
        endptindices = np.where(deglist[:, 1] == 1)
        # Look which of the endpoints lies most western, take that as the source for DFS traversal
        west = float('inf')
        source = deglist[0, 0]
        for i in list(endptindices[0]):
            westthis = nodesdict[str(deglist[i, 0])]["lon"]
            if westthis < west:
                source = deglist[i, 0]
                west = westthis

        component = []
        dfsedges = list(nx.dfs_edges(ccs[c], source))
        nend = dfsedges[0][0]
        path = [str(nend)]
        for e in dfsedges:
            if e[0] == nend: # path grows
                path.append(str(e[1]))
            else: # new path
                component.append(path)
                path = [str(e[0]), str(e[1])]
            nend = e[1]    
        component.append(path) # last path
        components.append(component)
    
    if "name" in way["tags"]:
        trackname = way["tags"]["name"]["name"]
    else:
        trackname = ""
    
    if "tracks" in way["tags"]:
        tracknum = int(way["tags"]["tracks"]["tracks"])
    else:
        tracknum = 1
    
    output = {"tags":{"name":trackname, "length": tracklength, "tracks": tracknum, "railway":way["tags"]["railway"]["railway"]}, "components":components, "nodes":nodesdict, "ways":[elemways[k]["id"] for k in range(len(elemways))]}
    return output

## Active tracks

In [ ]:
railtracks.drop()  
railtracks_error = []
numtracks = {mode:0 for mode in modes}
j = 1
for mode in modes:
    k = 0
    # No service, or if service then crossover (since they are used for active trains)
    for i,way in enumerate(ways_raw.find({"$or": [{"$and": [{"tags.railway.railway":mode}, {"tags.service.service": "crossover"}]}, {"$and": [{"tags.railway.railway":mode}, {"tags.service.service": {"$exists": False}}]}]})):
        try:
            res = assembleRailTrack(ways_raw, ways_derived, nodes_raw, way["_id"])
            res["_id"] = j
            j += 1
            railtracks.update_one({"_id":res["_id"]}, {"$set":res}, upsert=True)
            k += 1
        except:
            railtracks_error.append(way["_id"])
    numtracks[mode] = k
    print("Inserted " + str(numtracks[mode]) + " " +str(mode)+ " tracks in " + cityname)

with open("logs/" + cityname +"_railtrackserror.txt", "w") as f: 
    for s in railtracks_error:
        f.write("%s\n" % s)

## Passive tracks (parking)

In [ ]:
railtracksparking.drop()  
railtracksparking_error = []
numtracks = {mode:0 for mode in modes}
j = 1
for mode in modes:
    k = 0
    # Only service except for crossover (since they are used for active trains)
    for i,way in enumerate(ways_raw.find({"$and": [{"tags.railway.railway":mode}, {"tags.service.service": {"$ne": "crossover"}}, {"tags.service.service": {"$exists": True}}]})):
        try:
            res = assembleRailTrack(ways_raw, ways_derived, nodes_raw, way["_id"])
            res["_id"] = j
            j += 1
            railtracksparking.update_one({"_id":res["_id"]}, {"$set":res}, upsert=True)
            k += 1
        except:
            railtracksparking_error.append(way["_id"])
    numtracks[mode] = k
    print("Inserted " + str(numtracks[mode]) + " " +str(mode)+ " parking tracks in " + cityname)

with open("logs/" + cityname +"_railtracksparkingerror.txt", "w") as f: 
    for s in railtracksparking_error:
        f.write("%s\n" % s)